# Create a SageMaker MLOps Project for Pipelines
Note:  This requires that you have enabled products within SageMaker Studio

![](../img/enable-service-catalog-portfolio-for-studio.png)

In [ ]:
import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd
from pprint import pprint

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
sc = boto3.Session().client(service_name='servicecatalog', region_name=region)
sts = boto3.Session().client(service_name='sts', region_name=region)
iam = boto3.Session().client(service_name='iam', region_name=region)

In [ ]:
search_response = sc.search_products(
   Filters={
       'FullTextSearch': 
       [
           'MLOps template for model building, training, and deployment'
       ]
   }
)

sagemaker_pipeline_product_id = search_response['ProductViewSummaries'][0]['ProductId']
print(sagemaker_pipeline_product_id)

In [ ]:
describe_response = sc.describe_product(Id=sagemaker_pipeline_product_id)

sagemaker_pipeline_product_provisioning_artifact_id = describe_response['ProvisioningArtifacts'][0]['Id']

In [ ]:
print(sagemaker_pipeline_product_provisioning_artifact_id)

# Create a SageMaker Project


In [ ]:
import time

timestamp = int(time.time())

In [ ]:
sagemaker_project_name = 'dsoaws-{}'.format(timestamp)

create_response = sm.create_project(
    ProjectName=sagemaker_project_name,
    ProjectDescription='dsoaws-{}'.format(timestamp),
    ServiceCatalogProvisioningDetails={
        'ProductId': sagemaker_pipeline_product_id,
        'ProvisioningArtifactId': sagemaker_pipeline_product_provisioning_artifact_id
     }
)

sagemaker_project_id = create_response['ProjectId']
sagemaker_project_arn = create_response['ProjectArn']

print('Project ID {}'.format(sagemaker_project_id))
print('Project ARN {}'.format(sagemaker_project_arn))

In [ ]:
sagemaker_project_name_and_id = '{}-{}'.format(sagemaker_project_name, sagemaker_project_id)

print('Combined Project ID and ARN combined: {}'.format(sagemaker_project_name_and_id))

# _Wait for the Project to be Created_

In [ ]:
%%time

import time

try:
    describe_project_response = sm.describe_project(ProjectName=sagemaker_project_name)
    project_status = describe_project_response['ProjectStatus']
    print('Creating Project...')

    while project_status in ['Pending', 'CreateInProgress']:
        print('Please wait...')
        time.sleep(30)
        describe_project_response = sm.describe_project(ProjectName=sagemaker_project_name)
        project_status = describe_project_response['ProjectStatus']
        print('Project status: {}'.format(project_status))

    if project_status == 'CreateCompleted':   
        print('Project {}'.format(project_status))

    else:
        print('Project status: {}'.format(project_status))
        raise Exception('Project not created.')
        
except Exception as e:
    print(e)
    
print(describe_project_response)

# _Wait for Project to be Created ^^ Above ^^_

# Attach IAM Policies for FeatureStore 
This is used for Code Build Pipeline Executions.

In [ ]:
sc_role_name='AmazonSageMakerServiceCatalogProductsUseRole'

In [ ]:
account_id = sts.get_caller_identity()['Account']
print(account_id)

In [ ]:
response = iam.attach_role_policy(
    RoleName=sc_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
)

print(response)

In [ ]:
response = iam.attach_role_policy(
    RoleName=sc_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFeatureStoreAccess'
)

print(response)

# Stop the `Abalone` Sample Pipeline that Ships with SageMaker Pipelines
The sample "abalone" pipeline starts automatically when we create the project.  We want to stop this pipeline to release these resources and use them for our own pipeline.

In [ ]:
sample_abalone_pipeline_execution_arn = sm.list_pipeline_executions(PipelineName=sagemaker_project_name_and_id)['PipelineExecutionSummaries'][0]['PipelineExecutionArn']

print(sample_abalone_pipeline_execution_arn)

In [ ]:
sm.stop_pipeline_execution(PipelineExecutionArn=sample_abalone_pipeline_execution_arn)

In [ ]:
%%time

try:
    describe_pipeline_execution_response = sm.describe_pipeline_execution(PipelineExecutionArn=sample_abalone_pipeline_execution_arn)
    pipeline_execution_status = describe_pipeline_execution_response['PipelineExecutionStatus']

    while pipeline_execution_status not in ['Stopped', 'Failed']:
        print('Please wait...')
        time.sleep(30)
        describe_pipeline_execution_response = sm.describe_pipeline_execution(PipelineExecutionArn=sample_abalone_pipeline_execution_arn)
        pipeline_execution_status = describe_pipeline_execution_response['PipelineExecutionStatus']
        print('Pipeline execution status: {}'.format(pipeline_execution_status))

    if pipeline_execution_status in ['Stopped', 'Failed']:   
        print('Pipeline execution status {}'.format(pipeline_execution_status))
    else:
        print('Pipeline execution status: {}'.format(pipeline_execution_status))
        raise Exception('Pipeline execution not deleted.')
        
except Exception as e:
    print(e)
    
print(describe_pipeline_execution_response)

In [ ]:
sm.delete_pipeline(PipelineName=sagemaker_project_name_and_id)

# Clone the MLOps Repositories in AWS CodeCommit

In [ ]:
import os

sm_studio_root_path='/root/' 
sm_notebooks_root_path='/home/ec2-user/SageMaker/'

root_path = sm_notebooks_root_path if os.path.isdir(sm_notebooks_root_path) else sm_studio_root_path

print(root_path)

In [ ]:
print(region)

In [ ]:
code_commit_repo1 = 'https://git-codecommit.{}.amazonaws.com/v1/repos/sagemaker-{}-modelbuild'.format(region, sagemaker_project_name_and_id)
print(code_commit_repo1)

In [ ]:
sagemaker_mlops_build_code = '{}{}/sagemaker-{}-modelbuild'.format(root_path, sagemaker_project_name_and_id, sagemaker_project_name_and_id)
print(sagemaker_mlops_build_code)

In [ ]:
code_commit_repo2 = 'https://git-codecommit.{}.amazonaws.com/v1/repos/sagemaker-{}-modeldeploy'.format(region, sagemaker_project_name_and_id)
print(code_commit_repo2)

In [ ]:
sagemaker_mlops_deploy_code = '{}{}/sagemaker-{}-modeldeploy'.format(root_path, sagemaker_project_name_and_id, sagemaker_project_name_and_id)
print(sagemaker_mlops_deploy_code)

In [ ]:
!git config --global credential.helper '!aws codecommit credential-helper $@'
!git config --global credential.UseHttpPath true

# _Wait for Project to be Created ^^ Above ^^_

In [ ]:
!git clone $code_commit_repo1 $sagemaker_mlops_build_code

In [ ]:
!git clone $code_commit_repo2 $sagemaker_mlops_deploy_code

# Remove Sample `Abalone` Example Code

In [ ]:
!rm -rf $sagemaker_mlops_build_code/pipelines/abalone

# Copy Workshop Code Into Local Project Folders

In [ ]:
workshop_project_build_code='{}workshop/10_pipeline/sagemaker/sagemaker-project-modelbuild'.format(root_path)
print(workshop_project_build_code)

In [ ]:
workshop_project_deploy_code='{}workshop/10_pipeline/sagemaker/sagemaker-project-modeldeploy'.format(root_path)
print(workshop_project_deploy_code)

In [ ]:
!cp -R $workshop_project_build_code/* $sagemaker_mlops_build_code/

In [ ]:
!cp -R $workshop_project_deploy_code/* $sagemaker_mlops_deploy_code/

# Commit New Code 

In [ ]:
print(sagemaker_mlops_build_code)

In [ ]:
!cd $sagemaker_mlops_build_code; git status; git add --all .; git commit -m "Data Science on AWS"; git push

In [ ]:
!cd $sagemaker_mlops_deploy_code; git status; git add --all .; git commit -m "Data Science on AWS"; git push

# Store the Variables

In [ ]:
%store sagemaker_mlops_build_code
%store sagemaker_mlops_deploy_code
%store sagemaker_project_name
%store sagemaker_project_id
%store sagemaker_project_name_and_id
%store sagemaker_project_arn
%store sagemaker_pipeline_product_id
%store sagemaker_pipeline_product_provisioning_artifact_id

In [ ]:
!ls -al $sagemaker_mlops_build_code/pipelines/dsoaws/

In [ ]:
!pygmentize $sagemaker_mlops_build_code/pipelines/dsoaws/pipeline.py

# Wait for Pipeline Execution to Start
Now that we have committed code, our pipeline will start.  Let's wait for the pipeline to start.


In [ ]:
%%time

import time
from pprint import pprint

while True:
    try:
        print('Listing executions for our pipeline...')
        list_executions_response = sm.list_pipeline_executions(PipelineName=sagemaker_project_name_and_id)['PipelineExecutionSummaries']
        break;
    except Exception as e:
        print('Please wait...')
        time.sleep(30)   
    
pprint(list_executions_response)

In [ ]:
%store